## Read Vertical Speed

Uses model to open and (attempts to) predict the location of an instrument on screen

Models trained here are currently _very_ inaccurate. I'm also not 100% sure if I'm mapping the normalized values back to pixel coordinates correctly, as doing this with the formula you'd expect gives complete garbage.

The magic values in predict_bounding_box are an attempt to figure out what this formula should be. It's literally cheating by multiplying the normalized values so that they give the correct bounding box at the beginning of one of my test videos. While it seems to hold the bounding box consistently with no camera pan/zoom, once the camera transformations begin the model's accuracy plummets.

tl;dr - this didn't work

TODO: Make this work on video, output GIF of read values.

## Imports

In [1]:
import cv2
import numpy as np
import tensorflow as tf

## Read Vertical Speed

Currently only works on a single file

In [2]:
model = tf.keras.models.load_model('../../models/instrument_identifier.h5')

scale_factor = 0.5

def preprocess_frame(frame):
    # Resize the frame to match the model's input size
    resized_frame = cv2.resize(frame, (int(frame.shape[1] * scale_factor), int(frame.shape[0] * scale_factor)))

    # Normalize pixel values to [0, 1]
    resized_frame = resized_frame / 255.0

    # Add batch dimension
    preprocessed_frame = np.expand_dims(resized_frame, axis=0)
    
    return preprocessed_frame

def predict_bounding_box(frame):
    preprocessed_image = preprocess_frame(frame)

    predictions = model.predict(preprocessed_image)

    # Extract x, y, w, h from the predictions (adjust indices based on your model output)
    x, y, w, h = predictions[0]

    # Optionally, you may want to rescale the bounding box coordinates back to the original frame size
    original_width = 960
    original_height = 540

    # Expected answers:
    # 1060.0,650.0,130.0,120.0
    # X: 3.3928067684173584 | Y: 2.114502429962158 | Width: 0.3248142600059509 | Height: 0.220475971698761
    # 312, 307, 400, 544

    # Magic numbers based on calibration values
    x *= 312
    y *= 307
    w *= 400
    h *= 544

    
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)

    print(f"X: {x} | Y: {y} | Width: {w} | Height: {h}")

    

    return x, y, w, h


In [3]:
# Hacky means of making sure we don't go into infinite loop
def exit_application():
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.waitKey(1)
    cv2.waitKey(1)

In [5]:
input_file_path = '../../media/input_video/dynamic_camera/training_2.mp4'

cap = cv2.VideoCapture(input_file_path)

while True:
    ret, frame = cap.read()

    if not ret:
        break  # Break the loop if no more frames are available

    x, y, w, h = predict_bounding_box(frame)


    # cv2.putText(frame, f'Vertical Speed: {vertical_speed}', (150, 200), cv2.FONT_HERSHEY_PLAIN, 4.0, (0, 255, 0), 2)
    cv2.addWeighted(frame, 0.5, cv2.rectangle(frame.copy(), (x, y), (x + w, y + h), (0, 255, 0), -1), 0.5, 0, frame)

    # Display the current frame
    cv2.imshow('Frame', frame)

    # Wait for user input (e.g., a key press)
    key = cv2.waitKey(0)

    # Check if the user pressed the 'q' key to exit
    if key == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
exit_application()

1/1 [==============================] - 0s 50ms/step
X: 832 | Y: 519 | Width: 146 | Height: 141
1/1 [==============================] - 0s 57ms/step
X: 831 | Y: 520 | Width: 145 | Height: 139
1/1 [==============================] - 0s 57ms/step
X: 830 | Y: 519 | Width: 145 | Height: 138
1/1 [==============================] - 0s 47ms/step
X: 827 | Y: 518 | Width: 144 | Height: 138
1/1 [==============================] - 0s 48ms/step
X: 824 | Y: 517 | Width: 145 | Height: 137
1/1 [==============================] - 0s 48ms/step
X: 823 | Y: 516 | Width: 145 | Height: 138
1/1 [==============================] - 0s 48ms/step
X: 820 | Y: 516 | Width: 144 | Height: 136
1/1 [==============================] - 0s 51ms/step
X: 819 | Y: 515 | Width: 145 | Height: 136
1/1 [==============================] - 0s 49ms/step
X: 818 | Y: 515 | Width: 144 | Height: 134
1/1 [==============================] - 0s 48ms/step
X: 815 | Y: 514 | Width: 144 | Height: 132
1/1 [==============================] - 0s 47ms/ste